In [35]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import SABRModel
from python_module.tools import maximize_with_bounds

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [81]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Pricing parameters
F     = 5000
alpha = 0.25
beta  = 1.0
rho   = -0.
nu    = 1.5
r     = 0
bd    = 20
T     = bd / 250

# Scenario parameters
slide_to_compute = list(np.round(np.linspace(start=-0.3, stop=0.3, num=61), 2))

# Option portfolio
market_data_list = list()

# Option ATM
K = F
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

# Option -30
K = F * 0.71
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

# Option +30
K = F * 1.29
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

# Option -20
K = F * 0.81
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

# Option +20
K = F * 1.19
option_type = 'call' if K >= F else 'put'
market_pricing_results = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type, slide_list=slide_to_compute)
market_data_list.append({
    'symbol': f"{bd}T_{K}K_{option_type}",
    'option_type': option_type, 
    'time_to_maturity': bd, 
    'F':F, 'r': r, 'K': K, 'T': T, 'K/F': (K/F) - 1 ,**market_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

In [82]:
market_data_df

,option_type,time_to_maturity,F,r,K,T,K/F,IV,price,delta,gamma,vega,theta,vanna,volga,-0.30,-0.29,-0.28,-0.27,-0.26,-0.25,-0.24,-0.23,-0.22,-0.21,-0.20,-0.19,-0.18,-0.17,-0.16,-0.15,-0.14,-0.13,-0.12,-0.11,-0.10,-0.09,-0.08,-0.07,-0.06,-0.05,-0.04,-0.03,-0.02,-0.01,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20T_5000K_call,call,20,5000,0,"5,000.00",0.08,0.00,0.25,143.13,0.51,0.00,5.64,-3.58,0.06,-2.86,628.36,602.66,576.96,551.26,525.57,499.89,474.22,448.58,422.95,397.37,371.83,346.36,320.98,295.71,270.61,245.72,221.12,196.90,173.15,150.04,127.71,106.38,86.26,67.62,50.72,35.84,23.26,13.21,5.91,1.48,0.00,1.46,5.79,12.84,22.40,34.26,48.15,63.82,81.02,99.51,119.10,139.58,160.80,182.62,204.93,227.63,250.65,273.92,297.39,321.03,344.79,368.66,392.60,416.62,440.68,464.79,488.93,513.10,537.29,561.49,585.71
20T_3550.0K_put,put,20,5000,0,"3,550.00",0.08,-0.29,0.36,0.04,-0.00,0.00,0.01,-0.01,-0.01,46.54,127.55,101.20,78.98,60.71,46.05,34.53,25.65,18.91,13.86,10.11,7.36,5.34,3.87,2.79,2.01,1.45,1.03,0.73,0.51,0.35,0.24,0.16,0.10,0.06,0.03,0.01,-0.00,-0.01,-0.01,-0.00,0.00,0.01,0.01,0.02,0.03,0.04,0.06,0.07,0.08,0.09,0.10,0.12,0.13,0.14,0.16,0.17,0.18,0.20,0.21,0.22,0.24,0.25,0.26,0.28,0.29,0.30,0.32,0.33,0.34,0.36,0.37
20T_6450.0K_call,call,20,5000,0,"6,450.00",0.08,0.29,0.32,0.38,0.00,0.00,0.12,-0.10,0.08,304.22,3.89,3.75,3.61,3.47,3.32,3.18,3.04,2.90,2.76,2.61,2.47,2.33,2.19,2.05,1.91,1.77,1.63,1.49,1.35,1.21,1.08,0.94,0.81,0.68,0.56,0.44,0.33,0.23,0.14,0.06,0.00,-0.04,-0.04,-0.01,0.07,0.22,0.44,0.75,1.19,1.79,2.58,3.60,4.91,6.59,8.70,11.34,14.62,18.65,23.57,29.52,36.66,45.17,55.19,66.91,80.48,96.03,113.70,133.57,155.71,180.13,206.84
20T_4050.0000000000005K_put,put,20,5000,0,"4,050.00",0.08,-0.19,0.30,0.89,-0.01,0.00,0.25,-0.19,-0.14,495.70,543.59,495.64,448.40,402.12,357.09,313.67,272.27,233.31,197.20,164.30,134.88,109.08,86.92,68.25,52.82,40.31,30.33,22.49,16.42,11.79,8.29,5.68,3.77,2.39,1.42,0.75,0.32,0.07,-0.05,-0.06,0.00,0.12,0.28,0.48,0.70,0.95,1.21,1.48,1.76,2.04,2.34,2.63,2.93,3.23,3.54,3.85,4.15,4.46,4.77,5.08,5.39,5.71,6.02,6.33,6.64,6.95,7.27,7.58,7.89,8.21,8.52
20T_5950.0K_call,call,20,5000,0,"5,950.00",0.08,0.19,0.29,2.76,0.02,0.00,0.65,-0.47,0.34,"1,008.18",25.62,24.67,23.73,22.78,21.84,20.89,19.95,19.00,18.06,17.11,16.17,15.23,14.29,13.35,12.41,11.48,10.55,9.62,8.70,7.79,6.89,6.00,5.14,4.29,3.48,2.70,1.98,1.33,0.76,0.31,0.00,-0.13,-0.03,0.37,1.12,2.33,4.09,6.53,9.78,14.01,19.39,26.11,34.35,44.32,56.21,70.20,86.43,105.03,126.07,149.60,175.60,204.03,234.78,267.74,302.74,339.63,378.23,418.34,459.79,502.42,546.06


In [83]:
atm_market_data = market_data_df.loc[market_data_df['F']==market_data_df['K']]
atm_qty = -50000 / atm_market_data['theta']
atm_scenario = atm_market_data.loc[:, ['theta'] + slide_to_compute]
atm_scenario = atm_scenario.drop(0, axis=1)
atm_scenario.rename(columns={'theta': 0}, inplace=True)
atm_scenario = atm_scenario.sort_index(axis=1)
atm_scenario = atm_scenario.multiply(atm_qty, axis=0)

In [84]:
target_hedge_portfolio_scenario = atm_scenario*-1
otm_market_data = market_data_df.loc[market_data_df['F']!=market_data_df['K']]

In [85]:
target_scenario = [-0.3, -0.2, 0.2, 0.3]

In [86]:
X = otm_market_data[target_scenario].transpose().values
b = target_hedge_portfolio_scenario[target_scenario].iloc[0].values

w = np.linalg.solve(X, b)

weights = pd.Series(w, index=otm_market_data.index)

print(weights)


symbol
20T_3550.0K_put               112,047.83
20T_6450.0K_call               70,048.94
20T_4050.0000000000005K_put   -41,021.26
20T_5950.0K_call              -40,964.49
dtype: float64


In [87]:
otm_scenario = otm_market_data.loc[:, ['theta'] + slide_to_compute]
otm_scenario = otm_scenario.drop(0, axis=1)
otm_scenario.rename(columns={'theta': 0}, inplace=True)
otm_scenario = otm_scenario.sort_index(axis=1)
otm_scenario = otm_scenario.multiply(weights, axis=0)

In [88]:
total_scenario = atm_scenario.sum() + otm_scenario.sum()

In [89]:
px.line(total_scenario)

In [5]:
slide_df = market_data_df.loc[:, [-0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05]] 
theta = market_data_df['theta']
theta.name = 0
scenarios = pd.concat([slide_df, theta], axis=1)
scenarios = scenarios.transpose().sort_index().transpose()

In [6]:
scenarios

,-0.05,-0.04,-0.03,-0.02,-0.01,0.00,0.01,0.02,0.03,0.04,0.05
symbol,,,,,,,,,,,
20T_4339K_put,5.14,2.62,1.02,0.15,-0.15,-0.74,0.49,1.25,2.22,3.34,4.59
20T_4564K_put,15.71,8.80,4.08,1.23,-0.04,-1.71,1.12,3.10,5.78,9.01,12.68
20T_4716K_put,25.99,15.42,7.76,2.82,0.32,-2.57,1.56,4.71,9.21,14.82,21.33
20T_4828K_put,32.49,20.03,10.63,4.25,0.77,-3.15,1.71,5.63,11.50,19.03,27.97
20T_4926K_put,35.59,22.62,12.50,5.35,1.20,-3.48,1.64,5.95,12.69,21.61,32.45
20T_5000K_call,35.84,23.26,13.21,5.91,1.48,-3.58,1.46,5.79,12.84,22.40,34.26
20T_5104K_call,33.33,22.19,13.04,6.14,1.73,-3.48,1.09,5.06,11.92,21.59,33.95
20T_5206K_call,28.60,19.44,11.75,5.77,1.78,-3.16,0.67,3.97,10.04,18.93,30.66
20T_5324K_call,22.06,15.30,9.50,4.86,1.61,-2.62,0.26,2.65,7.37,14.62,24.53


In [7]:
target_scenario = -0.05
df = scenarios.drop(target_scenario, axis=1)
s = scenarios[target_scenario]

In [8]:
# Constrain weights between -0.5 and 0.8
w_opt, res = maximize_with_bounds(df=df, s=s, w_min=-10000, w_max=10000)

print("Optimal weights:\n", w_opt)
print("Maximized objective (w^T s):", float(w_opt.dot(s)))

Optimal weights:
 symbol
20T_4339K_put    -10,000.00
20T_4564K_put    -10,000.00
20T_4716K_put    -10,000.00
20T_4828K_put      7,829.54
20T_4926K_put     10,000.00
20T_5000K_call    10,000.00
20T_5104K_call    10,000.00
20T_5206K_call    -9,312.29
20T_5324K_call   -10,000.00
20T_5516K_call   -10,000.00
20T_5815K_call   -10,000.00
Name: weights, dtype: float64
Maximized objective (w^T s): 175775.99706054755


In [9]:
market_data_df.loc[:, [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]].multiply(w_opt, axis=0).sum(axis=0)

-0.30    -8,767,113.32
-0.10       281,511.61
-0.05       175,776.00
-0.04       112,895.34
-0.03        58,695.76
-0.02        19,793.98
-0.01             0.00
0.01         17,358.70
0.02         46,827.37
0.03         80,899.64
0.04        110,526.93
0.05        125,893.69
0.10       -325,206.25
0.30    -13,771,373.84
dtype: float64

In [14]:
market_data_df.loc[:, [-0.3, -0.1, -0.05, -0.04, -0.03, -0.02, -0.01, 'theta', 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.3]].loc['20T_5000K_call']*10_000

-0.30    6,283,630.31
-0.10    1,277,115.38
-0.05      358,406.13
-0.04      232,579.61
-0.03      132,131.11
-0.02       59,064.61
-0.01       14,787.91
 theta     -35,767.74
0.01        14,641.39
0.02        57,914.54
0.03       128,368.92
0.04       224,034.80
0.05       342,583.56
0.10     1,190,964.49
0.30     5,857,122.45
Name: 20T_5000K_call, dtype: float64

In [11]:
market_data_df['vega']* w_opt

symbol
20T_4339K_put    -10,537.45
20T_4564K_put    -25,747.21
20T_4716K_put    -39,795.96
20T_4828K_put     38,579.96
20T_4926K_put     54,772.12
20T_5000K_call    56,382.64
20T_5104K_call    54,681.28
20T_5206K_call   -45,872.46
20T_5324K_call   -40,355.02
20T_5516K_call   -25,451.95
20T_5815K_call   -10,215.32
dtype: float64